In [18]:
import os
import glob
import shutil
from strands import Agent, tool
from strands_tools import file_write 

import os
os.environ['BYPASS_TOOL_CONSENT'] = 'true'

print("Cell 1: Global imports successful. ✅")

Cell 1: Global imports successful. ✅


In [19]:
# --- Agent Memory / Dependency Tracker ---
class DependencyTracker:
    """Manages the translated context (memory) for dependency awareness."""
    def __init__(self):
        self.translated_context = {}

    @tool
    def track_and_get_context(self, current_file_path: str, translated_content: str = None) -> str:
        """Stores a snippet of translated content and retrieves necessary context."""
        if translated_content:
            self.translated_context[current_file_path] = translated_content[:200] + "..."
            return f"Context updated for {current_file_path}. Now translating next file."

        if not self.translated_context:
            return "No prior translation context available. Starting fresh."

        context_list = [f"Translated file {k}: {v}" for k, v in self.translated_context.items()]
        return "Available context from previous translations:\n" + "\n".join(context_list)
        
print("Cell 2: DependencyTracker class defined.")

Cell 2: DependencyTracker class defined.


In [20]:
# --- Tool for Dynamic Dependency Management ---
@tool
def update_build_script(project_dir: str, dependency_name: str, dependency_version: str = "1.3.2", configuration: str = "implementation") -> str:
    """
    Appends a new dependency to a dummy 'build.gradle.kts' file for tracking purposes.
    """
    build_file_path = os.path.join(project_dir, 'build.gradle.kts')
    os.makedirs(project_dir, exist_ok=True)
    
    new_dependency_line = f'{configuration}("org.jetbrains.kotlinx:{dependency_name}:{dependency_version}")'
    
    # Create or append to a dummy build file for visibility
    if not os.path.exists(build_file_path):
        initial_content = "dependencies {\n    // Initial dependencies here\n}\n"
        with open(build_file_path, 'w') as f:
            f.write(initial_content)
    
    try:
        with open(build_file_path, 'r') as f:
            content = f.read()
        
        if new_dependency_line not in content:
            with open(build_file_path, 'a') as f:
                f.write(f"\n{new_dependency_line}\n")
            return f"SUCCESS: Logged dependency {dependency_name} to build script."
        
        return f"INFO: Dependency {dependency_name} already logged."
        
    except Exception as e:
        return f"FAILURE: Could not update build script log: {e}"

print("Cell 3: 'update_build_script' tool defined. 📝")


Cell 3: 'update_build_script' tool defined. 📝


In [21]:
# --- NEW TOOL: Read File Content ---
@tool
def file_read(file_path: str) -> str:
    """Reads and returns the complete content of the file at the given path."""
    try:
        with open(file_path, 'r') as f:
            return f.read()
    except FileNotFoundError:
        return f"Error: File not found at path: {file_path}"
    except Exception as e:
        return f"Error reading file: {e}"

# NOTE: Ensure this tool and its dependencies are included in the execution of your current Cell 1.

In [22]:
# --- Tool for Setting up the Full, Correct Gradle Structure ---
@tool
def setup_gradle_project(project_dir: str, main_class_name: str) -> str:
    """
    Creates the essential build.gradle.kts and settings.gradle.kts files 
    with the correct structure for a Kotlin application, including serialization.
    """
    # 1. settings.gradle.kts (Defines the project name)
    settings_content = f'rootProject.name = "{os.path.basename(project_dir)}"'
    with open(os.path.join(project_dir, 'settings.gradle.kts'), 'w') as f:
        f.write(settings_content)

    # 2. build.gradle.kts (Defines the required plugins, repos, and application settings)
    build_content = f"""
plugins {{
    kotlin("jvm") version "1.9.22" 
    kotlin("plugin.serialization") version "1.9.22"
    application 
}}

group = "com.example.app"
version = "1.0.0"

repositories {{
    mavenCentral()
}}

sourceSets {{
    main {{
        kotlin.srcDir("src")
    }}
}}

dependencies {{
    implementation("org.jetbrains.kotlin:kotlin-stdlib-jdk8")
    
    // Testing dependencies 
    testImplementation("org.junit.jupiter:junit-jupiter-api:5.10.1")
    testRuntimeOnly("org.junit.jupiter:junit-jupiter-engine:5.10.1")
}}

application {{
    mainClass.set("{main_class_name}")
}}

kotlin {{
    jvmToolchain(17) 
}}
"""
    with open(os.path.join(project_dir, 'build.gradle.kts'), 'w') as f:
        f.write(build_content)

    # 3. Create the simplified source structure 
    os.makedirs(os.path.join(project_dir, 'src'), exist_ok=True)
    os.makedirs(os.path.join(project_dir, 'test'), exist_ok=True)
    
    return "Full, corrected Gradle structure created successfully with necessary blocks."

print("New Cell: setup_gradle_project tool defined with complete Gradle boilerplate. 🛠️")

New Cell: setup_gradle_project tool defined with complete Gradle boilerplate. 🛠️


In [23]:
# Assuming this is placed in your tools definition cell (e.g., Cell 3 in the modularized structure)

import subprocess # Ensure this is imported in Cell 1

@tool
def add_gradle_wrapper(project_dir: str) -> str:
    """
    Runs the 'gradle wrapper' command in the project directory to generate 
    the gradlew and gradlew.bat scripts and required files.
    """
    try:
        print("\n*** Executing Gradle Wrapper Setup ***")
        # Run the 'wrapper' task, which creates gradlew and its properties file
        result = subprocess.run(
            ['gradle', 'wrapper'], 
            cwd=project_dir, 
            capture_output=True, 
            text=True,
            check=True # Raise exception on non-zero exit code
        )
        print("*** Gradle Wrapper Successful ***\n")
        return f"SUCCESS: Gradle wrapper files generated in {project_dir}. Output: {result.stdout[-100:]}"
    except subprocess.CalledProcessError as e:
        return f"FAILURE: Gradle wrapper generation failed. Log: {e.stderr}"
    except FileNotFoundError:
        return "ERROR: 'gradle' command not found. Ensure Gradle is installed and in your system PATH."

print("New Tool: 'add_gradle_wrapper' defined. 🎁")

New Tool: 'add_gradle_wrapper' defined. 🎁


In [24]:
# --- Configuration ---
# Set this to the name of the folder containing your Python source files
source_directory = 'my_large_project' 
# Set this to the name of the folder where the Kotlin source and tests will be saved
target_directory = 'kotlin_package_with_tests' 
# Extensions the agent should process for translation
files_to_translate = ['.py', '.pyi'] 

# --- Initialize Global Instance ---
dependency_tracker = DependencyTracker()

print(f"Cell 4: Configuration variables set and DependencyTracker instance created. ")

Cell 4: Configuration variables set and DependencyTracker instance created. 


In [25]:
def run_translation_and_test_generation(input_dir: str, output_dir: str, file_extensions_to_translate: list, tracker_instance: DependencyTracker):
    """
    Orchestrates the file-by-file translation, test generation, and dependency logging.
    """
    
    # 1. Setup Environment
    if os.path.exists(output_dir):
        print(f"Warning: Deleting existing output directory '{output_dir}'")
        shutil.rmtree(output_dir)
        
    # Define dedicated subdirectories for output files
    kotlin_src_dir = os.path.join(output_dir, 'src')
    kotlin_test_dir = os.path.join(output_dir, 'test')
    os.makedirs(kotlin_src_dir, exist_ok=True)
    os.makedirs(kotlin_test_dir, exist_ok=True) 
    
    all_files = glob.glob(os.path.join(input_dir, '**', '*'), recursive=True)
    file_paths = [f for f in all_files if os.path.isfile(f)]
    
    if not file_paths:
        print(f"Error: No files found in '{input_dir}'")
        return

    # 2. Instantiate the Agent (Tools and System Prompt are set here)
    agent = Agent(
        tools=[tracker_instance.track_and_get_context, file_write, update_build_script],
        #auto_approve_tool_calls=True,
        system_prompt=(
            "You are an expert Python-to-Kotlin translator and unit test generator with dependency management duties. "
            "For every file, your tasks are: "
            "1. You MUST assume the project structure is already set up (settings.gradle.kts, build.gradle.kts exists)." # Changed assumption
            "2. When translating, if you use a Kotlin library (e.g., kotlinx.serialization), you MUST first use the 'update_build_script' tool "
            "to log the required dependency before using 'file_write'. Use '1.3.2' as the version for 'kotlinx-serialization-json'."
            "3. Translate the Python code to idiomatic Kotlin (saving to 'src/')."
            "4. Generate a comprehensive JUnit 5 test suite for the translated code (saving to 'test/')."
            "5. You MUST use 'file_write' TWICE: once for the main Kotlin file, and once for the test file."
            "6. Ensure code is Java 17 compatible."
        )
    )

    print(f"Cell 5: Agent instantiated. Total files to process: {len(file_paths)}")
    print("-" * 30)

    # ⭐️ NEW ORCHESTRATION STEP: Call the setup tool directly before the loop
    # This ensures the build.gradle.kts file is created before the agent tries to update it.
    main_class_dummy = "com.example.MainKt" # Use a placeholder
    setup_gradle_project(output_dir, main_class_dummy) 
    print(f"Initial Gradle files created in {output_dir}.")
    print("-" * 30)
    
    # 3. Execution Loop
    for source_path in file_paths:
        relative_path = os.path.relpath(source_path, input_dir)
        source_ext = os.path.splitext(source_path)[1].lower()
        
        is_code = source_ext in file_extensions_to_translate
        
        if is_code:
            main_kotlin_path = os.path.join(kotlin_src_dir, relative_path).replace(source_ext, '.kt')
            test_kotlin_path = os.path.join(kotlin_test_dir, relative_path).replace(source_ext, 'Test.kt')
            
            os.makedirs(os.path.dirname(main_kotlin_path), exist_ok=True)
            os.makedirs(os.path.dirname(test_kotlin_path), exist_ok=True)
            action = "TRANSLATE+TEST+DEPS"
        else:
            main_kotlin_path = os.path.join(output_dir, relative_path)
            action = "COPY"

        print(f"-> Processing [{action}]: {relative_path}")

        # --- Handle Non-Code Files (Direct Copy) ---
        if not is_code:
            shutil.copy2(source_path, main_kotlin_path)
            print(f"   (Copied asset directly to: {main_kotlin_path})")
            continue

        # --- Handle Code Files (Agentic Translation + Test Generation) ---
        try:
            with open(source_path, 'r') as f:
                file_content = f.read()

            prompt = (
                f"File to translate: '{relative_path}'. "
                f"Translate the following Python code to Kotlin, saving to '{main_kotlin_path}'. "
                f"Also, generate a comprehensive JUnit 5 test suite for the translated code, saving it to '{test_kotlin_path}'. "
                f"Source Content:\n```python\n{file_content}\n```"
                f"The project root for dependency management is '{output_dir}'. Use tools for context and dependency logging."
            )

            agent(prompt) 
            print(f"<- Finished: {relative_path}\n")
            
        except Exception as e:
            print(f"!!! Error processing {relative_path}: {e}")
            
    print("-" * 30)
    print(f"\n✅ Translation and Test Generation complete. Check the '{output_dir}' directory.")

print("Cell 5: Main orchestration function defined.  orchestrator 🎼")


Cell 5: Main orchestration function defined.  orchestrator 🎼


In [26]:
# Execute the function defined in Cell 5 using variables from Cell 4
run_translation_and_test_generation(
    input_dir=source_directory, 
    output_dir=target_directory, 
    file_extensions_to_translate=files_to_translate,
    tracker_instance=dependency_tracker 
)

print("\nCell 6: Execution complete. Check logs above. 🚀")


Cell 5: Agent instantiated. Total files to process: 3
------------------------------
Initial Gradle files created in kotlin_package_with_tests.
------------------------------
-> Processing [TRANSLATE+TEST+DEPS]: call_weather_api.py
I'll help you translate the Python code to Kotlin and generate comprehensive tests. Let me start by tracking the context and then proceed with the translation.
Tool #1: track_and_get_context
Now I'll add the required JSON serialization dependency and translate the code to Kotlin:
Tool #2: update_build_script

Tool #3: file_write


╔═════════════════ File Write Operation ══════════════════╗
║                                                         ║
║ Path: kotlin_package_with_tests\src\call_weather_api.kt ║
║ Size: 2762 characters                                   ║
║                                                         ║
╚═════════════════════════════════════════════════════════╝

╔═══════════════════════════════ Write Successful ═══════════════════════════════╗
║ File written successfully to kotlin_package_with_tests\src\call_weather_api.kt ║
╚════════════════════════════════════════════════════════════════════════════════╝

Now I'll create a comprehensive JUnit 5 test suite:
Tool #4: file_write


╔════════════════════ File Write Operation ════════════════════╗
║                                                              ║
║ Path: kotlin_package_with_tests\test\call_weather_apiTest.kt ║
║ Size: 12519 characters                                       ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝

...

╔════════════════════════════ Write Successful ═════════════════════════════╗
║ File written successfully to kotlin_package_with_tests\src\weather_api.kt ║
╚═══════════════════════════════════════════════════════════════════════════╝

Now I'll create a comprehensive test suite:
Tool #12: file_write


╔═════════════════ File Write Operation ══════════════════╗
║                                                         ║
║ Path: kotlin_package_with_tests\test\weather_apiTest.kt ║
║ Size: 25376 characters                                  ║
║                                                         ║
╚═════════════════════════════════════════════════════════╝

╔═══════════════════════════════ Write Successful ═══════════════════════════════╗
║ File written successfully to kotlin_package_with_tests\test\weather_apiTest.kt ║
╚════════════════════════════════════════════════════════════════════════════════╝

Perfect! I've successfully translated the `weather_api.py` file to Kotlin and created a comprehensive test suite. Here's what I've accomplished:

## Translation Summary:

### 1. **Main Kotlin File** (`src\weather_api.kt`):

**Key Features:**
- **WeatherApi class**: Modern Kotlin implementation using Ktor HTTP client
- **Coroutine support**: Async/await pattern with `suspend` functions
- **Backward compatibility**: Synchronous wrappers and global function matching original Python signature
- **Type-safe data classes**: Complete weather API response structure with proper serialization
- **Error handling**: Graceful handling of network errors and HTTP status codes
- **Resource management**: Proper HTTP client lifecycle management

**API Methods:**
- `getCurrentWeather()` - Async version using coroutines
- `getCurrentWeatherSync()` - Synchronous wrapper for compatibility
- `getCurrentWeatherAsMap()` - Returns Map<String, Any> like Python dict
- Global `getCurrentWeather()` function - Exact

In [28]:
# Assuming 'target_directory' is defined and the Agent definition setup is complete

# --- Agent 2: Refactoring and Configuration Fixer ---
print("\n\n#####################################################")
print("## AGENT 2: REFACTORING & CONFIGURATION FIXER")
print("#####################################################")

# Instantiate the Refactoring Agent
refactoring_agent = Agent(
    # Agent needs to read files, write files, and know about dependency logging (for error analysis)
    # The update_build_script tool (defined previously) will now handle the Coroutines dependency
    tools=[file_read, file_write, update_build_script], 
    #auto_approve_tool_calls=True,
    system_prompt=(
        "You are Agent 2, an expert Kotlin and Gradle Refactoring Specialist. Your task is to resolve structural and configuration issues. "
        "1. You MUST use 'file_read' to inspect files and 'file_write' to correct them."
        "2. Resolve compilation issues like duplicate functions by deleting the mock implementation."
        "3. **Dependency Fix (Coroutines):** If the translated code uses 'kotlinx.coroutines.runBlocking', you MUST call 'update_build_script' to add the 'kotlinx-coroutines-core' dependency (version 1.7.3) using the 'implementation' configuration."
        "4. Correct dependency placement in 'build.gradle.kts' by reading the file and rewriting it with dependencies inside the correct 'dependencies { }' block."
        "5. **Main Class Fix:** Correct the 'mainClass' entry in 'build.gradle.kts' to the fully qualified name: 'com.example.Call_weather_apiKt'."
        "6. Add the appropriate 'package com.example' declaration to ALL translated .kt files."
    )
)

# A comprehensive prompt listing all the issues Agent 1 missed and the new fixes
refactoring_prompt = f"""
Fix all of these issues sequentially in the project directory '{target_directory}':

1. **Fix Duplicate Function:** Read the content of '{target_directory}/call_weather_api.kt' and remove the mock/placeholder function named 'getCurrentWeather()'.
2. **Fix Dependency Placement:** Read '{target_directory}/build.gradle.kts'. Rewrite the entire file, ensuring ALL dependency lines (e.g., implementation("org.jetbrains.kotlinx:ktor-client-core:2.3.2")) are moved INSIDE the dependencies {{ }} block.
3. **Fix Main Class:** Read '{target_directory}/build.gradle.kts'. Find the 'mainClass.set' line and change the value to the fully qualified name: "com.example.Call_weather_apiKt".
4. **Add Coroutines Dependency:** Use the 'update_build_script' tool to add the 'kotlinx-coroutines-core' dependency (version 1.7.3) to the project.
5. **Fix Package Structure:** Add the line "package com.example" as the very first line to the following files:
   - '{target_directory}/src/weather_api.kt'
   - '{target_directory}/src/call_weather_api.kt'

Execute these fixes using your tools.
"""

refactoring_agent(refactoring_prompt)

print("\nCell: Refactoring Agent execution complete with final build fixes. ✅")



#####################################################
## AGENT 2: REFACTORING & CONFIGURATION FIXER
#####################################################
I'll fix all these issues sequentially. Let me start by examining the files and then applying the necessary fixes.
Tool #1: file_read
Let me check the actual file structure:
Tool #2: file_read
Now let me check the weather_api.kt file:
Tool #3: file_read
Now let me check the build.gradle.kts file:
Tool #4: file_read
Now I'll fix all the issues sequentially:

## 1. Fix Duplicate Function in call_weather_api.kt
I can see there are actually two different `getCurrentWeather` functions - one in the WeatherApi object (mock) and one as a global function in weather_api.kt. I need to remove the mock one from call_weather_api.kt:
Tool #5: file_write


╔═════════════════ File Write Operation ══════════════════╗
║                                                         ║
║ Path: kotlin_package_with_tests/src/call_weather_api.kt ║
║ Size: 1300 characters                                   ║
║                                                         ║
╚═════════════════════════════════════════════════════════╝

╔═══════════════════════════════ Write Successful ═══════════════════════════════╗
║ File written successfully to kotlin_package_with_tests/src/call_weather_api.kt ║
╚════════════════════════════════════════════════════════════════════════════════╝

## 2. Fix Package Structure for weather_api.kt
Tool #6: file_write


╔═══════════════ File Write Operation ═══════════════╗
║                                                    ║
║ Path: kotlin_package_with_tests/src/weather_api.kt ║
║ Size: 8751 characters                              ║
║                                                    ║
╚════════════════════════════════════════════════════╝

╔════════════════════════════ Write Successful ═════════════════════════════╗
║ File written successfully to kotlin_package_with_tests/src/weather_api.kt ║
╚═══════════════════════════════════════════════════════════════════════════╝

## 3. Fix Dependency Placement and Main Class in build.gradle.kts
Tool #7: file_write


╔══════════════ File Write Operation ══════════════╗
║                                                  ║
║ Path: kotlin_package_with_tests/build.gradle.kts ║
║ Size: 1059 characters                            ║
║                                                  ║
╚══════════════════════════════════════════════════╝

╔═══════════════════════════ Write Successful ════════════════════════════╗
║ File written successfully to kotlin_package_with_tests/build.gradle.kts ║
╚═════════════════════════════════════════════════════════════════════════╝

## 4. Add Coroutines Dependency (already done in build.gradle.kts, but let's use the tool as requested)
Tool #8: update_build_script
Perfect! All the fixes have been applied successfully:

## Summary of Changes Made:

1. **✅ Fixed Duplicate Function**: Removed the mock `WeatherApi` object and its `getCurrentWeather()` function from `call_weather_api.kt`. The file now uses the global `getCurrentWeather()` function from `weather_api.kt`.

2. **✅ Fixed Dependency Placement**: Moved ALL dependency lines inside the `dependencies { }` block in `build.gradle.kts`. All implementation dependencies are now properly contained within the dependencies block.

3. **✅ Fixed Main Class**: Changed the `mainClass.set` value from `"com.example.MainKt"` to `"com.example.Call_weather_apiKt"` to match the fully qualified name of the main function.

4. **✅ Added Coroutines Dependency**: The `kotlinx-coroutines-core` dependency (version 1.7.3) was already properly placed in the dependencies block during the r

In [ ]:
# Don't execute this block for now as the path for gradle is not set correctly. First need to set the path for gradle correctly

# Assuming the Refactoring Agent (Cell 7) has finished successfully

# --- Agent 3: Wrapper Creator ---
print("\n\n#####################################################")
print("## AGENT 3: GRADLE WRAPPER CREATOR")
print("#####################################################")

# Instantiate the Wrapper Agent
wrapper_agent = Agent(
    # Agent only needs the wrapper tool
    tools=[add_gradle_wrapper],
    #auto_approve_tool_calls=True,
    system_prompt=(
        "You are Agent 3, the packaging finalizer. Your sole task is to ensure the project can be built anywhere. "
        "You MUST call the 'add_gradle_wrapper' tool to generate the necessary build scripts in the project directory."
    )
)

# Execution prompt
wrapper_prompt = f"The code is refactored. Use 'add_gradle_wrapper' to generate the gradlew scripts in the '{target_directory}' folder."

wrapper_agent(wrapper_prompt)

print("\nCell 8: Wrapper Agent execution complete. Project is now fully self-contained! ✅")